# Project 3 + 4

In this project, you will perform a logistic regression on the admissions data we've been working with in projects 1 and 2.

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import pylab as pl
import numpy as np


In [2]:
df_raw = pd.read_csv("../assets/admissions.csv")
df = df_raw.dropna() 
print df.head()

   admit    gre   gpa  prestige
0      0  380.0  3.61       3.0
1      1  660.0  3.67       3.0
2      1  800.0  4.00       1.0
3      1  640.0  3.19       4.0
4      0  520.0  2.93       4.0



## Part 1. Frequency Tables

#### 1. Let's create a frequency table of our variables

In [3]:
# frequency table for prestige and whether or not someone was admittedb


print pd.crosstab([df.gre],df.admit).head(), pd.crosstab([df.gpa],df.admit).head()

admit  0  1
gre        
220.0  1  0
300.0  2  1
340.0  3  1
360.0  4  0
380.0  8  0 admit  0  1
gpa        
2.26   1  0
2.42   1  1
2.48   1  0
2.52   1  0
2.55   1  0


In [4]:
pd.crosstab([df.gpa],df.admit).head()

admit,0,1
gpa,,
2.26,1,0
2.42,1,1
2.48,1,0
2.52,1,0
2.55,1,0


In [5]:
pd.crosstab( [df.prestige], df.admit)

admit,0,1
prestige,,
1.0,28,33
2.0,95,53
3.0,93,28
4.0,55,12


## Part 2. Return of dummy variables

#### 2.1 Create class or dummy variables for prestige 

In [6]:
dummy_ranks = pd.get_dummies(data = df['prestige'],prefix = 'prestige', drop_first = False)

dummy_ranks.head()

,prestige_1.0,prestige_2.0,prestige_3.0,prestige_4.0
0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0
2,1.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,1.0


#### 2.2 When modeling our class variables, how many do we need? 



Answer: 3. The fourth column is implied.

## Part 3. Hand calculating odds ratios

Develop your intuition about expected outcomes by hand calculating odds ratios.

In [7]:
cols_to_keep = ['admit', 'gre', 'gpa']
handCalc = df[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_1':])
print handCalc.head()

   admit    gre   gpa  prestige_1.0  prestige_2.0  prestige_3.0  prestige_4.0
0      0  380.0  3.61           0.0           0.0           1.0           0.0
1      1  660.0  3.67           0.0           0.0           1.0           0.0
2      1  800.0  4.00           1.0           0.0           0.0           0.0
3      1  640.0  3.19           0.0           0.0           0.0           1.0
4      0  520.0  2.93           0.0           0.0           0.0           1.0


In [8]:
#crosstab prestige 1 admission 
# frequency table cutting prestige and whether or not someone was admitted

pd.crosstab(handCalc['prestige_1.0'],handCalc.admit)

admit,0,1
prestige_1.0,,
0.0,243,93
1.0,28,33


In [9]:
#### 3.1 Use the cross tab above to calculate the odds of being admitted to grad school if you attended a #1 ranked college

In [10]:
#p1_admitted = handCalc[(handCalc['prestige_2.0'] != 1) & (handCalc['admit'] == 1) & (handCalc['prestige_3.0'] != 1) & (handCalc['prestige_4.0'] != 1)]
#p1_denied = handCalc[(handCalc['prestige_2.0'] != 1) & (handCalc['admit'] == 0) & (handCalc['prestige_3.0'] != 1) & (handCalc['prestige_4.0'] != 1)]

p1_odds = 33.0/28


#### 3.2 Now calculate the odds of admission if you did not attend a #1 ranked college

In [11]:
p1_else_odds = 93.0/243

#### 3.3 Calculate the odds ratio

In [12]:
p1_odds_ratio = p1_odds / p1_else_odds
print p1_odds_ratio

3.07949308756


#### 3.4 Write this finding in a sentenance: 

Answer: You're 3.07X more likely to be admitted to a grad school if you addented a school with a prestige rank of 1 than if you did not.

#### 3.5 Print the cross tab for prestige_4

In [13]:
pd.crosstab(handCalc['prestige_4.0'],handCalc.admit)

admit,0,1
prestige_4.0,,
0.0,216,114
1.0,55,12


In [14]:

p4_odds = 12.0/55
p4_else_odds = 114.0/216

#### 3.6 Calculate the OR 

In [15]:
p4_odds_ratio = p4_odds / p4_else_odds
print p4_odds_ratio

0.413397129187


#### 3.7 Write this finding in a sentence

Answer: Students from prestige 4 schools have 41% of the chance of being admitted that non prstige 4 students would have.

## Part 4. Analysis

In [16]:
# create a clean data frame for the regression
cols_to_keep = ['admit', 'gre', 'gpa']
data = df[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_2':])
print data.head()

   admit    gre   gpa  prestige_2.0  prestige_3.0  prestige_4.0
0      0  380.0  3.61           0.0           1.0           0.0
1      1  660.0  3.67           0.0           1.0           0.0
2      1  800.0  4.00           0.0           0.0           0.0
3      1  640.0  3.19           0.0           0.0           1.0
4      0  520.0  2.93           0.0           0.0           1.0


We're going to add a constant term for our Logistic Regression. The statsmodels function we're going to be using requires that intercepts/constants are specified explicitly.

In [17]:
# manually add the intercept
data['intercept'] = 1.0

#### 4.1 Set the covariates to a variable called train_cols

In [18]:
train_cols = ['gre','gpa','prestige_2.0','prestige_3.0','prestige_4.0']

#### 4.2 Fit the model

In [19]:
logit = sm.Logit(data['admit'],data[train_cols])

result = logit.fit()

Optimization terminated successfully.
         Current function value: 0.589121
         Iterations 5


#### 4.3 Print the summary results

In [20]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  admit   No. Observations:                  397
Model:                          Logit   Df Residuals:                      392
Method:                           MLE   Df Model:                            4
Date:                Tue, 30 Aug 2016   Pseudo R-squ.:                 0.05722
Time:                        09:30:16   Log-Likelihood:                -233.88
converged:                       True   LL-Null:                       -248.08
                                        LLR p-value:                 1.039e-05
================================================================================
                   coef    std err          z      P>|z|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------
gre              0.0014      0.001      1.308      0.191        -0.001     0.003
gpa             -0.1323      0.195     -0.680      0.497        -0.514     0.249
prestige_2.0    -0.9562      0.302     -3.171      0.002        -1.547    -0.365
prestige_3.0    -1.5375      0.332     -4.627      0.000        -2.189    -0.886
prestige_4.0    -1.8699      0.401     -4.658      0.000        -2.657    -1.083
================================================================================
"""

In [21]:
print result.conf_int()

                     0         1
gre          -0.000680  0.003414
gpa          -0.513657  0.249045
prestige_2.0 -1.547279 -0.365166
prestige_3.0 -2.188769 -0.886230
prestige_4.0 -2.656743 -1.083112


#### 4.4 Calculate the odds ratios of the coeffiencents and their 95% CI intervals

hint 1: np.exp(X)

hint 2: conf['OR'] = params
        
           conf.columns = ['2.5%', '97.5%', 'OR']

In [22]:
print np.exp(result.params)

gre             1.001368
gpa             0.876073
prestige_2.0    0.384342
prestige_3.0    0.214918
prestige_4.0    0.154135
dtype: float64


#### 4.5 Interpret the OR of Prestige_2

Answer: For someone from a prestige 2 school, we can expect the odds of being admitted to be reduced by ~62% from a prestige 1 school.

#### 4.6 Interpret the OR of GPA

Answer: 

## Part 5: Predicted probablities


As a way of evaluating our classifier, we're going to recreate the dataset with every logical combination of input values. This will allow us to see how the predicted probability of admission increases/decreases across different variables. First we're going to generate the combinations using a helper function called cartesian (above).

We're going to use np.linspace to create a range of values for "gre" and "gpa". This creates a range of linearly spaced values from a specified min and maximum value--in our case just the min/max observed values.

In [23]:
def cartesian(arrays, out=None):
    """
    Generate a cartesian product of input arrays.
    Parameters
    ----------
    arrays : list of array-like
        1-D arrays to form the cartesian product of.
    out : ndarray
        Array to place the cartesian product in.
    Returns
    -------
    out : ndarray
        2-D array of shape (M, len(arrays)) containing cartesian products
        formed of input arrays.
    Examples
    --------
    >>> cartesian(([1, 2, 3], [4, 5], [6, 7]))
    array([[1, 4, 6],
           [1, 4, 7],
           [1, 5, 6],
           [1, 5, 7],
           [2, 4, 6],
           [2, 4, 7],
           [2, 5, 6],
           [2, 5, 7],
           [3, 4, 6],
           [3, 4, 7],
           [3, 5, 6],
           [3, 5, 7]])
    """

    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype

    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)

    m = n / arrays[0].size
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in xrange(1, arrays[0].size):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
    return out

In [24]:
# instead of generating all possible values of GRE and GPA, we're going
# to use an evenly spaced range of 10 values from the min to the max 
gres = np.linspace(data['gre'].min(), data['gre'].max(), 10)
print gres
# array([ 220.        ,  284.44444444,  348.88888889,  413.33333333,
#         477.77777778,  542.22222222,  606.66666667,  671.11111111,
#         735.55555556,  800.        ])
gpas = np.linspace(data['gpa'].min(), data['gpa'].max(), 10)
print gpas
# array([ 2.26      ,  2.45333333,  2.64666667,  2.84      ,  3.03333333,
#         3.22666667,  3.42      ,  3.61333333,  3.80666667,  4.        ])


# enumerate all possibilities
combos = pd.DataFrame(cartesian([gres, gpas, [1, 2, 3, 4], [1.]]))

[ 220.          284.44444444  348.88888889  413.33333333  477.77777778
  542.22222222  606.66666667  671.11111111  735.55555556  800.        ]
[ 2.26        2.45333333  2.64666667  2.84        3.03333333  3.22666667
  3.42        3.61333333  3.80666667  4.        ]


#### 5.1 Recreate the dummy variables

In [28]:
# recreate the dummy variables

combos.columns = ['gre', 'gpa', 'prestige', 'intercept']
dummy_ranks = pd.get_dummies(data = combos['prestige'], prefix = 'prestige')
dummy_ranks.columns = ['prestige_1','prestige_2','prestige_3','prestige_4']
# keep only what we need for making predictions



cols_to_keep = ['gre','gpa','prestige','intercept']
combos = combos[cols_to_keep].join(dummy_ranks.ix[:'prestige_2':])


ValueError: Length mismatch: Expected axis has 8 elements, new values have 4 elements

#### 5.2 Make predictions on the enumerated dataset

In [29]:
combos['admit_pred'] = result.predict(combos[train_cols])

KeyError: "['prestige_2.0' 'prestige_3.0' 'prestige_4.0'] not in index"

#### 5.3 Interpret findings for the last 4 observations

Answer: 

## Bonus

Plot the probability of being admitted into graduate school, stratified by GPA and GRE score.